In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

'''----------------------------------------------------------------------------------------'''

with open('anna.txt','r') as f:
    text = f.read()
text[:100]


'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [2]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [3]:
encoded[:100]

array([32, 64, 71, 36, 63,  1, 29,  5, 44, 33, 33, 33, 42, 71, 36, 36, 59,
        5, 10, 71, 35, 39, 74, 39,  1, 53,  5, 71, 29,  1,  5, 71, 74, 74,
        5, 71, 74, 39, 65,  1, 38,  5,  1, 27,  1, 29, 59,  5, 28, 25, 64,
       71, 36, 36, 59,  5, 10, 71, 35, 39, 74, 59,  5, 39, 53,  5, 28, 25,
       64, 71, 36, 36, 59,  5, 39, 25,  5, 39, 63, 53,  5,  4, 66, 25, 33,
       66, 71, 59, 46, 33, 33,  0, 27,  1, 29, 59, 63, 64, 39, 25])

In [4]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot
if False:
    def one_hot_encode(arr, n_labels):

        # Initialize the the encoded array
        #print(arr.shape)
        print(np.multiply(*arr.shape))
        one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
        print("array one hot shape")
        print(one_hot.shape[0]) 
        print("-------------------------------------------------------")
        print(arr.flatten())
        print("--------------------------------------------------------")
        print("one hot",one_hot)
        print("[[0,1,2],[3,5,1]]")

        print( one_hot[np.arange(one_hot.shape[0]), arr.flatten()])

        # Fill the appropriate elements with ones
        one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
        print(one_hot)

        # Finally reshape it to get back to the original array
        one_hot = one_hot.reshape((*arr.shape, n_labels))

        return one_hot

In [5]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [6]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [7]:
batches = get_batches(encoded,50,8)
x,y = next(batches)

In [8]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[32 64 71 36 63  1 29  5]
 [29  5 51  4 46 33 33 45]
 [41 71 74 74  1 22  5 71]
 [64  1  5 53 63 29 71 25]
 [ 5 27  4 22 65 71 12  5]
 [74  5  4 10  5 71  5 36]
 [ 1  5 53 71 39 22 12  5]
 [ 4 25  5 71 25 22  5 40]
 [51  5  4 10  5 63 64  1]
 [71 63  5 53 63 71 39 29]]

y
 [[64 71 36 63  1 29  5 44]
 [ 5 51  4 46 33 33 45 31]
 [71 74 74  1 22  5 71 74]
 [ 1  5 53 63 29 71 25 51]
 [27  4 22 65 71 12  5 71]
 [ 5  4 10  5 71  5 36 39]
 [ 5 53 71 39 22 12  5 71]
 [25  5 71 25 22  5 40 59]
 [ 5  4 10  5 63 64  1  5]
 [63  5 53 63 71 39 29 41]]


In [9]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print("cuda is available")
else:
    print("training on cpu ")

cuda is available


In [10]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [13]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y).long()
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    targets = targets.long()
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [11]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(76, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=76, bias=True)
)


In [14]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2391... Val Loss: 3.2153
Epoch: 1/20... Step: 20... Loss: 3.1613... Val Loss: 3.1432
Epoch: 1/20... Step: 30... Loss: 3.1309... Val Loss: 3.1325
Epoch: 1/20... Step: 40... Loss: 3.1202... Val Loss: 3.1296
Epoch: 1/20... Step: 50... Loss: 3.1134... Val Loss: 3.1270
Epoch: 1/20... Step: 60... Loss: 3.1002... Val Loss: 3.1236
Epoch: 1/20... Step: 70... Loss: 3.1092... Val Loss: 3.1163
Epoch: 2/20... Step: 80... Loss: 3.0984... Val Loss: 3.1012
Epoch: 2/20... Step: 90... Loss: 3.0673... Val Loss: 3.0763
Epoch: 2/20... Step: 100... Loss: 2.9926... Val Loss: 3.0025
Epoch: 2/20... Step: 110... Loss: 2.8847... Val Loss: 2.9044
Epoch: 2/20... Step: 120... Loss: 2.7581... Val Loss: 2.7557
Epoch: 2/20... Step: 130... Loss: 2.6702... Val Loss: 2.6593
Epoch: 2/20... Step: 140... Loss: 2.5850... Val Loss: 2.5635
Epoch: 3/20... Step: 150... Loss: 2.5349... Val Loss: 2.5061
Epoch: 3/20... Step: 160... Loss: 2.4915... Val Loss: 2.4507
Epoch: 3/20... Step: 170... Loss:

Epoch: 20/20... Step: 1340... Loss: 1.3856... Val Loss: 1.2881
Epoch: 20/20... Step: 1350... Loss: 1.3968... Val Loss: 1.2845
Epoch: 20/20... Step: 1360... Loss: 1.3200... Val Loss: 1.2856
Epoch: 20/20... Step: 1370... Loss: 1.3753... Val Loss: 1.2854
Epoch: 20/20... Step: 1380... Loss: 1.3709... Val Loss: 1.2797
Epoch: 20/20... Step: 1390... Loss: 1.3962... Val Loss: 1.2790
Epoch: 20/20... Step: 1400... Loss: 1.4201... Val Loss: 1.2805


In [27]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [22]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [23]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [19]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna, she was standing all the meaton and treished that sort
in this some, who had
certainly he seined to the marriage as the sones, had been
straight on, and all that he went all at the position, and short there was no too her hortes which
though he's a from him with a children and the sectnate time of the saming of his. "It were if a meadow, I did not sungrittent and the strong what was
no since that's not a shand is time. I didn't see, to do. And so that I could never me so true."

"What's why to do?" answered Anna had been to say to him, and there was
an that in her servage to still
all this was the plooded without thinking, that he succeeded a sickly
started tinnersely.

"I don't know it and though it must himself to take you was not in the prince as too things that I'm
tried on that, I'll tell you that to me her and I sold to me her, and shall never more that it was a long always a sould will, think of his wife,
and he has say," he asked that he had not havid state, but he had no

In [28]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open(model_name, 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [29]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said that whom he heard the constion went to be a simple still as in some the wors and a studing often and sont of suck her eyes that this with a sturge into her hands. She
was
coushed
to herself how she was a letter, he was
the come with his face on the peasant
servent with her.

"Well, to that's it?"

"No,, and wantted to her."

"What went to be a sincal of mose are," he said, and who had been the back to to make
him
to say he without
spite, and his
choored has a partly and takes with supper, and had been
delanitaliar was to be so that
though he
was at it.

"And I'v not could so like the mose as a crande stain outs."

"No, I, it said that..." he said, and tried to
bo at an englasing of any horeself were tightly as a land of a bell to his brother the same was starking that has brought all his wife, souched her sorts, and
there saw all her attonciously
shooking over hards
which all his strange to the proncess hid
before the recoldection with a brown,
bus all
the same she was 